In [1]:
import pandas as pd

path = "/Users/angelavansprang/Documents/PhD/transformers for time series/Autoformer/dataset/traffic/traffic.csv"
df_raw = pd.read_csv(path)

df_raw

,date,0,1,2,3,4,5,6,7,8,...,852,853,854,855,856,857,858,859,860,OT
0,2016-07-01 02:00:00,0.0048,0.0146,0.0289,0.0142,0.0064,0.0232,0.0162,0.0242,0.0341,...,0.0051,0.0051,0.0074,0.0079,0.0051,0.0051,0.0339,0.0051,0.0100,0.0121
1,2016-07-01 03:00:00,0.0072,0.0148,0.0350,0.0174,0.0084,0.0240,0.0201,0.0338,0.0434,...,0.0036,0.0036,0.0107,0.0058,0.0036,0.0036,0.0348,0.0036,0.0087,0.0136
2,2016-07-01 04:00:00,0.0040,0.0101,0.0267,0.0124,0.0049,0.0170,0.0127,0.0255,0.0332,...,0.0030,0.0030,0.0043,0.0050,0.0030,0.0030,0.0327,0.0030,0.0061,0.0107
3,2016-07-01 05:00:00,0.0039,0.0060,0.0218,0.0090,0.0029,0.0118,0.0088,0.0163,0.0211,...,0.0033,0.0033,0.0019,0.0052,0.0033,0.0033,0.0292,0.0033,0.0040,0.0071
4,2016-07-01 06:00:00,0.0042,0.0055,0.0191,0.0082,0.0024,0.0095,0.0064,0.0087,0.0144,...,0.0049,0.0049,0.0011,0.0071,0.0049,0.0049,0.0264,0.0049,0.0040,0.0039
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17539,2018-07-01 21:00:00,0.0342,0.0332,0.1378,0.0559,0.0165,0.0386,0.0457,0.0687,0.0716,...,0.0139,0.0179,0.0278,0.0280,0.0227,0.0375,0.0142,0.0201,0.0341,0.0429
17540,2018-07-01 22:00:00,0.0285,0.0253,0.0542,0.0485,0.0131,0.0331,0.0372,0.0569,0.0669,...,0.0116,0.0155,0.0201,0.0267,0.0207,0.0320,0.0122,0.0163,0.0306,0.0363
17541,2018-07-01 23:00:00,0.0276,0.0257,0.0551,0.0474,0.0121,0.0326,0.0381,0.0566,0.0675,...,0.0109,0.0142,0.0208,0.0252,0.0189,0.0287,0.0123,0.0152,0.0292,0.0325
17542,2018-07-02 00:00:00,0.0232,0.0229,0.0496,0.0435,0.0119,0.0297,0.0361,0.0517,0.0598,...,0.0092,0.0124,0.0219,0.0215,0.0169,0.0258,0.0096,0.0130,0.0252,0.0279


In [ ]:
from models import Autoformer
from utils.tools import dotdict
import torch


def obtain_autoformer(pred_len, dataset):
    assert dataset in ["ECL", "Exchange", "ETTm2"]
    # assert pred_len in [96, 192, 336, 720]

    args = dotdict()
    args.pred_len = pred_len

    args.enc_in = 1
    args.dec_in = 1
    args.c_out = 1

    if dataset == "ETTm2":
        args.label_len = 96
    else:
        args.label_len = 48

    args.target = "OT"
    args.des = "train"
    args.dropout = 0.05
    args.num_workers = 10
    args.gpu = 0
    args.lradj = "type1"
    args.devices = "0"
    args.use_gpu = False
    args.use_multi_gpu = False
    args.freq = "h"
    args.checkpoints = "./checkpoints/"
    args.bucket_size = 4
    args.n_hashes = 4
    args.is_trainging = True
    args.data = "custom"
    args.features = "M"
    args.seq_len = 96
    # args.label_len = 48
    args.e_layers = 2
    args.d_layers = 1
    args.n_heads = 8
    args.factor = 1
    args.d_model = 512
    args.des = "Exp"
    args.itr = 1
    args.d_ff = 2048
    args.moving_avg = 25
    args.factor = 3
    args.distil = True
    args.output_attention = False
    args.embed = "timeF"

    autoformer_path = f"/Users/angelavansprang/Documents/PhD/transformers for time series/Autoformer/checkpoints/{dataset}_96_{pred_len}_Autoformer_{'ETTm2' if dataset == 'ETTm2' else 'custom'}_ft{'S' if dataset == 'ETTm2' else 'M'}_sl96_ll{'96' if dataset == 'ETTm2' else '48'}_pl{pred_len}_dm512_nh8_el2_dl1_df2048_fc3_ebtimeF_dtTrue_Exp_0/checkpoint.pth"

    autoformer = Autoformer.Model(args).float()
    autoformer.load_state_dict(
        torch.load(autoformer_path, map_location=torch.device("cpu"))
    )

    autoformer.eval()

    return autoformer

In [ ]:
from data_provider.data_loader import Dataset_Custom
from torch.utils.data import DataLoader


def obtain_data_loader_traffic(
    pred_len, seq_len=96, model="non-informer", split="test"
):
    label_len = 48

    if model == "informer":
        features = "S"
    else:
        features = "M"

    data_set_electricity = Dataset_Custom(
        root_path="/Users/angelavansprang/Documents/PhD/transformers for time series/Autoformer/dataset/traffic/",
        data_path="traffic.csv",
        flag=split,  # flag must be train in order for the linear dataloader to contain the same data
        size=[seq_len, label_len, pred_len],  # seq_len, label_len, pred_len
        features=features,
        target="OT",  # default
        timeenc=1,
        freq="h",
    )

    data_loader_electricity = DataLoader(
        data_set_electricity,
        batch_size=1,
        shuffle=False,
        num_workers=0,
        drop_last=False,
    )
    return data_loader_electricity